In [1]:
# Import
from dotenv import load_dotenv
import os
import sys
import requests 
from requests import post
import json
import time
import pandas as pd
import ast

In [2]:
load_dotenv()

client_id_igdb = os.getenv("IGDB_ID_CLIENT")
client_secret_igdb = os.getenv("IGDB_SECRET_CLIENT")

In [3]:
print(client_secret_igdb)
print(client_id_igdb)

t0bfpbrfjsxb4eqpt2kezg82p3uhsw
4pmce94am1hizog6ff9feucpmtki1q


In [4]:
# Récupération du token d'accès pour IGDB

url = f"https://id.twitch.tv/oauth2/token?client_id={client_id_igdb}&client_secret={client_secret_igdb}&grant_type=client_credentials"

print(url)
response = requests.post(url)
if response.status_code == 200:
    response_json = response.json()
    if 'expires_in' in response_json == 0:
        print(f"Le token a expiré, il faut le renouveler")
    access_token = response_json['access_token']
    print(f"Access Token: {access_token}")
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)

https://id.twitch.tv/oauth2/token?client_id=4pmce94am1hizog6ff9feucpmtki1q&client_secret=t0bfpbrfjsxb4eqpt2kezg82p3uhsw&grant_type=client_credentials
Access Token: izkz2s6b0cq4zdugcshw4m4024dbwn


In [5]:
# Récuperer le nombres de jeux
url_igdb_engines_count = "https://api.igdb.com/v4/games/count"

headers = {
    'Client-ID': client_id_igdb,   
    'Authorization': f'Bearer {access_token}',
    }

response = post(url_igdb_engines_count, headers=headers, data='fields *;')
print(response.json())

{'count': 323266}


In [6]:
# Récupération des games


headers_game = {
    'Client-ID': client_id_igdb,
    'Authorization': f'Bearer {access_token}',
}

response = post('https://api.igdb.com/v4/games', headers=headers_game , data='fields name,age_ratings,category,dlcs,first_release_date,game_engines,game_status,genres,language_supports,multiplayer_modes,videos,websites,total_rating,total_rating,summary  ; limit 500;')
# Vérification de la réponse
if response.status_code == 200:
    # On transforme la réponse en JSON
    response_json = response.json()
    # On transforme le JSON en DataFrame
    df_games = pd.DataFrame(response_json)
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)
    
df_games

,id,category,game_engines,genres,name,summary,videos,first_release_date,websites,language_supports,total_rating,age_ratings,multiplayer_modes,game_status,dlcs
0,330684,14,[439],"[10, 33]",Nightmare Kart: The Old Karts,An upcoming free expansion to Nightmare Kart w...,[139717],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,177310,0,[750],"[13, 32]",The Undying Beast,"There was a flash of light, a choir of glass, ...",NaN,1.586736e+09,"[237634, 473902]",NaN,NaN,NaN,NaN,NaN,NaN
2,282079,5,NaN,"[5, 8]",Rockman X: New Year 2023,A small hack to celebrate the Chinese New Year...,NaN,1.672531e+09,[606539],[876934],NaN,NaN,NaN,NaN,NaN
3,63844,0,NaN,[14],Ace wo Nerae!,A tennis game for the Super Famicom based on t...,[43074],7.565184e+08,"[161465, 373934]",[685891],52.904629,NaN,NaN,NaN,NaN
4,338541,7,NaN,[10],Mario Kart Tour: Vancouver Tour,NaN,NaN,1.581466e+09,[724679],[1205446],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,218403,0,[456],"[8, 31, 32]",Omega Warp,Omega Warp is a 2D sci-fi themed action-advent...,"[75133, 111316, 114101]",1.718755e+09,"[312819, 504288, 312820, 312821, 312822, 31282...","[489487, 489488]",NaN,[176009],NaN,NaN,NaN
496,102753,0,NaN,NaN,Blocky San Andreas Police 2018,NaN,NaN,NaN,[429634],NaN,NaN,NaN,NaN,NaN,NaN
497,191066,0,[64],[8],Monster Evolution,"Play as a monster. Create havoc, feed on human...",NaN,1.230768e+09,"[263668, 484198]",NaN,NaN,NaN,NaN,NaN,NaN
498,134276,0,[400],"[13, 32]",Unpacking,Unpacking is a job-simulator game about inspec...,[36530],1.536970e+09,"[142013, 778930, 142012]","[300737, 300738, 300739, 300740, 300741, 30074...",NaN,"[109241, 110652, 124898, 106263]",NaN,NaN,NaN


In [6]:
def get_all_games():
    all_games = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/games', headers=headers_game, data=f'fields name,first_release_date,total_rating,slug,summary; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            all_games.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(all_games)

In [8]:
df_games = get_all_games()

In [10]:
# On transforme first_release_date en date 
df_games['first_release_date'] = pd.to_datetime(df_games['first_release_date'], unit='s', errors='coerce').dt.strftime('%Y-%m-%d')

df_games.head(5)

,id,name,slug,summary,first_release_date,total_rating
0,330684,Nightmare Kart: The Old Karts,nightmare-kart-the-old-karts,An upcoming free expansion to Nightmare Kart w...,NaN,NaN
1,177310,The Undying Beast,the-undying-beast,"There was a flash of light, a choir of glass, ...",2020-04-13,NaN
2,282079,Rockman X: New Year 2023,rockman-x-new-year-2023,A small hack to celebrate the Chinese New Year...,2023-01-01,NaN
3,63844,Ace wo Nerae!,ace-wo-nerae,A tennis game for the Super Famicom based on t...,1993-12-22,52.904629
4,338541,Mario Kart Tour: Vancouver Tour,mario-kart-tour-vancouver-tour,NaN,2020-02-12,NaN


In [11]:
df_games.to_csv("../data-game/csv/games.csv", index=False)

In [15]:
fichier_source = r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv/game.csv'
taille_bloc = 100000  # Nombre de lignes par fichier (ajuste si besoin)

try:
    # Utilisez l'une des solutions ci-dessus. Ici, j'utilise 'on_bad_lines='skip''
    # pour permettre au code de s'exécuter même avec des lignes mal formées.
    # Si votre délimiteur est différent, ajoutez 'delimiter=';'' par exemple.
    df_reader = pd.read_csv(fichier_source, chunksize=taille_bloc, on_bad_lines='skip')

    for i, chunk in enumerate(df_reader):
        nom_fichier = f"donnees_part{i+1}.csv"
        chunk.to_csv(nom_fichier, index=False)
        print(f"Fichier créé : {nom_fichier}")

except pd.errors.ParserError as e:
    print(f"Une erreur de parsing est survenue : {e}")
    print("Veuillez vérifier le format de votre fichier CSV, en particulier la ligne indiquée dans l'erreur.")
    print("Assurez-vous que les délimiteurs sont cohérents et que les champs contenant des délimiteurs sont correctement entre guillemets.")
except FileNotFoundError:
    print(f"Erreur : Le fichier source '{fichier_source}' n'a pas été trouvé.")
except Exception as e:
    print(f"Une erreur inattendue est survenue : {e}")

Fichier créé : donnees_part1.csv
Fichier créé : donnees_part2.csv
Fichier créé : donnees_part3.csv
Fichier créé : donnees_part4.csv
Fichier créé : donnees_part5.csv


Plateform_version

In [ ]:
# Récuperer le nombres game_videos
url_igdb_engines_count = "https://api.igdb.com/v4/game_videos/count"

headers = {
    'Client-ID': client_id_igdb,   
    'Authorization': f'Bearer {access_token}',
    }

response = post(url_igdb_engines_count, headers=headers, data='fields *;')
print(response.json())

{'count': 118798}


In [ ]:
# Récupération des game_videos


headers_game = {
    'Client-ID': client_id_igdb,
    'Authorization': f'Bearer {access_token}',
}

response = post('https://api.igdb.com/v4/game_videos/', headers=headers_game , data=f'fields checksum,game,name,video_id; limit 500;')
# Vérification de la réponse
if response.status_code == 200:
    # On transforme la réponse en JSON
    response_json = response.json()
    # On transforme le JSON en DataFrame
    df_game_video = pd.DataFrame(response_json)
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)
    
df_game_video

,id,game,name,video_id,checksum
0,8402,18845,Zone D13 Fly Over Trailer,bPFJuANnahE,be0aa110-be58-e009-0cb6-2a3e55cc2a68
1,10414,18688,Release Trailer,Ky5mScjzYZk,21c8cda7-4b42-bd2d-0053-97f55cefe101
2,7866,18880,Trailer,qmtaQqgHfuo,ec60aacc-07ab-428c-4404-1817df5ee847
3,10429,1212,Alvin Battle Trailer,2qvOHnoS8BA,e233c00a-dc60-29c1-b7fd-c333b6600caf
4,10436,22281,Trailer,d0Dj0rLptQg,f469c14a-fd39-e76e-e6a5-29418a41fb8e
...,...,...,...,...,...
495,24741,109460,Trailer,nBqXSBxOH-A,d8bf1e6f-a32c-a0a8-f635-7b0bc571b46d
496,24748,36950,Trailer,oh2RHMcfHkg,af43eb69-0211-4a38-566f-8617be120604
497,24742,41061,Trailer,W15GB7M8FUc,5c5cec2c-58f3-405b-83b2-3da0532ad80d
498,30765,91823,Gameplay video,itKJoAM6Ze4,1e379bc1-2ce6-50c1-26a6-0d7fca08fdc2


In [32]:
def get_all_games_videos():
    get_all_games_video = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/game_videos/', headers=headers , data=f'fields checksum,game,name,video_id; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            get_all_games_video.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(get_all_games_video)

In [33]:
df_game_video = get_all_games_videos()

In [ ]:
# Exporter le DataFrame en CSV
df_game_video.to_csv("../data-game/csv", index=False)

COMPANY

In [ ]:
# Récuperer le nombres compagnies
url_igdb_engines_count = "https://api.igdb.com/v4/companies/count"

headers = {
    'Client-ID': client_id_igdb,   
    'Authorization': f'Bearer {access_token}',
    }

response = post(url_igdb_engines_count, headers=headers, data='fields *;')
print(response.json())

{'count': 60861}


In [7]:
# Récupération des compagnies


headers_game = {
    'Client-ID': client_id_igdb,
    'Authorization': f'Bearer {access_token}',
}

response = post('https://api.igdb.com/v4/companies', headers=headers , data=f'fields changed_company_id, country, description,developed,name,parent,published,slug,start_date,status, websites, logo; limit 500;')
# Vérification de la réponse
if response.status_code == 200:
    # On transforme la réponse en JSON
    response_json = response.json()
    # On transforme le JSON en DataFrame
    df_company = pd.DataFrame(response_json)
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)
    
df_company

,id,country,description,developed,name,slug,start_date,websites,status,logo,published,parent,changed_company_id
0,61668,840.0,CyrustheVirus Studios was established by Sheld...,[289276],CyrusTheVirus Studios,cyrusthevirus-studios,1.548979e+09,[13818],0,NaN,NaN,NaN,NaN
1,30720,203.0,300AD is an independent company associating de...,"[80245, 52452, 144481, 142326, 142325, 144511,...",300AD,300ad,NaN,[4031],0,3995.0,NaN,NaN,NaN
2,540,276.0,10TACLE STUDIOS was specialized in the develop...,[92755],10tacle Studios,10tacle-studios,1.072829e+09,NaN,1,1902.0,"[736, 738, 11286, 15762, 27466, 125168, 21516,...",NaN,NaN
3,64121,156.0,Sunborn Network Technology (formerly LBX Netwo...,NaN,Sunborn Network Technology,sunborn-network-technology,1.445731e+09,[14834],0,18002.0,NaN,17151.0,NaN
4,62963,76.0,NaN,[339674],NoShadow,noshadow,1.630454e+09,NaN,0,NaN,[339674],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,57747,NaN,We are a small group of developers who spend c...,"[264939, 264938]",FSDesigns,fsdesigns,NaN,[12429],0,14669.0,NaN,NaN,NaN
496,64164,NaN,NaN,[338688],Overscoped Games,overscoped-games,NaN,[14883],0,NaN,NaN,NaN,NaN
497,8224,276.0,"Aerosoft - The Simulation Company \n""Offering ...","[16945, 161797, 53362, 161850, 161861, 161853,...",Aerosoft GmbH,aerosoft-gmbh,NaN,[1768],0,1965.0,"[22438, 36304, 32829, 67299, 33738, 10594, 111...",NaN,NaN
498,62301,NaN,NaN,[334737],FrogJB,frogjb,NaN,NaN,0,NaN,[334737],NaN,NaN


In [8]:
def get_all_company():
    get_all_company = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/companies', headers=headers , data=f'fields changed_company_id, country, description,developed,name,parent,published,slug,start_date,status, websites, logo; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            get_all_company.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break

    return pd.DataFrame(get_all_company)

In [9]:
df_company = get_all_company()


In [10]:
#creation df_company_developed sur la base de company
df_company_developed = df_company.copy()
df_company_developed = df_company_developed.drop(columns=["country","description", "name", "slug","start_date", "websites", "status", "logo", "published", "parent", "changed_company_id"])
df_company_developed = df_company_developed.explode("developed")
df_company_developed 

,id,developed
0,61668,289276
1,30720,80245
1,30720,52452
1,30720,144481
1,30720,142326
...,...,...
61192,1277,8014
61192,1277,18244
61192,1277,26688
61192,1277,313290


In [11]:
df_company_developed.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv/company_developed.csv', index=False)

In [32]:
#on enelve les colonnes qui ne servent pas 
df_company = df_company.drop(columns=["developed", "websites", "logo", "published"])
# Exporter le DataFrame en CSV
df_company.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv/company.csv', index=False)

COMPANY_LOGO

In [53]:
# Récupération des logo compagnies


headers_game = {
    'Client-ID': client_id_igdb,
    'Authorization': f'Bearer {access_token}',
}

response = post('https://api.igdb.com/v4/company_logos', headers=headers , data=f'fields image_id,url; limit 500;')
# Vérification de la réponse
if response.status_code == 200:
    # On transforme la réponse en JSON
    response_json = response.json()
    # On transforme le JSON en DataFrame
    df_compagny_logo = pd.DataFrame(response_json)
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)
    
df_compagny_logo

,id,image_id,url
0,968,svplxuxn983cwmohg6w3,//images.igdb.com/igdb/image/upload/t_thumb/sv...
1,1972,cl1is,//images.igdb.com/igdb/image/upload/t_thumb/cl...
2,14346,clb2i,//images.igdb.com/igdb/image/upload/t_thumb/cl...
3,1543,iscz8jnzg9cddypn3nle,//images.igdb.com/igdb/image/upload/t_thumb/is...
4,1574,gwy6dwd3rp0ckgj4vqpx,//images.igdb.com/igdb/image/upload/t_thumb/gw...
...,...,...,...
495,1280,quhi9leoceglbzqpmrx4,//images.igdb.com/igdb/image/upload/t_thumb/qu...
496,311,uo6otpmodrdvlou3fhvn,//images.igdb.com/igdb/image/upload/t_thumb/uo...
497,1318,uwddahypagxdfn0l6ae6,//images.igdb.com/igdb/image/upload/t_thumb/uw...
498,1327,snhadznmkwgg4jscoy6z,//images.igdb.com/igdb/image/upload/t_thumb/sn...


In [21]:
def get_all_company_logos():
    get_all_company_logo = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/company_logos', headers=headers , data=f'fields url; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            get_all_company_logo.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break


    return pd.DataFrame(get_all_company_logo)

In [24]:
df_company_logos = get_all_company_logos()



In [26]:
df_company_logos.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv/company_logos.csv', index=False)

COMPANY status

In [44]:
# Récupération des statues de compagnies


headers_game = {
    'Client-ID': client_id_igdb,
    'Authorization': f'Bearer {access_token}',
}

response = post('https://api.igdb.com/v4/company_statuses', headers=headers , data=f'fields name; limit 500;')
# Vérification de la réponse
if response.status_code == 200:
    # On transforme la réponse en JSON
    response_json = response.json()
    # On transforme le JSON en DataFrame
    df_compagny_statues = pd.DataFrame(response_json)
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)
    
df_compagny_statues

,id,name
0,0,active
1,1,defunct
2,2,merged
3,3,renamed


In [45]:
df_compagny_statues.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv\company_status.csv', index=False)

COMPANY COMPANY STATUES

In [17]:
# Récupération des statues de compagnies


headers_game = {
    'Client-ID': client_id_igdb,
    'Authorization': f'Bearer {access_token}',
}

response = post('https://api.igdb.com/v4/companies', headers=headers , data=f'fields status; limit 500;')
# Vérification de la réponse
if response.status_code == 200:
    # On transforme la réponse en JSON
    response_json = response.json()
    # On transforme le JSON en DataFrame
    df_company_companies_statues  = pd.DataFrame(response_json)
else:
    print("Error fetching data from the API")
    print("Status code:", response.status_code)
    
df_company_companies_statues 

,id,status
0,61668,0
1,30720,0
2,540,1
3,64121,0
4,62963,0
...,...,...
495,57747,0
496,64164,0
497,8224,0
498,62301,0


In [16]:
def get_all_company_companies():
    get_all_company_companies = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/companies', headers=headers , data=f'fields status; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            get_all_company_companies.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break


    return pd.DataFrame(get_all_company_companies)

In [18]:
df_company_companies_statues = get_all_company_companies()

In [19]:
df_company_companies_statues

,id,status
0,61668,0.0
1,30720,0.0
2,540,1.0
3,64121,0.0
4,62963,0.0
...,...,...
61189,41907,0.0
61190,37730,0.0
61191,11925,0.0
61192,61707,0.0


In [20]:
df_company_companies_statues.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv\company_companies_statues.csv', index=False)

COMPANY_LOGO

In [27]:
def get_all_company_logo():
    get_all_company_logo = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/companies', headers=headers , data=f'fields logo; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            get_all_company_logo.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break


    return pd.DataFrame(get_all_company_logo)

In [28]:
df_company_logo = get_all_company_logo()
df_company_logo

,id,logo
0,61668,NaN
1,30720,3995.0
2,540,1902.0
3,64121,18002.0
4,62963,NaN
...,...,...
61189,41907,10007.0
61190,37730,12274.0
61191,11925,1181.0
61192,61707,NaN


In [29]:
df_company_logo.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv\company_logo.csv', index=False)

MULTIPLAYER_MODE

In [ ]:
def get_all_multiplayer_modes():
    get_all_multiplayer_modes = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/multiplayer_modes', headers=headers , data=f'fields campaigncoop,lancoop,offlinecoop,offlinecoopmax,offlinemax,onlinecoop,onlinecoopmax,onlinemax,platform,splitscreen,splitscreenonline; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            get_all_multiplayer_modes.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break


    return pd.DataFrame(get_all_multiplayer_modes)

In [32]:
df_multiplayer_modes = get_all_multiplayer_modes()
df_multiplayer_modes

,id,campaigncoop,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,offlinecoopmax,onlinecoopmax,onlinemax
0,9953,False,False,False,2.0,False,22.0,False,NaN,NaN,NaN
1,1832,True,False,True,0.0,False,12.0,False,2.0,0.0,0.0
2,7987,False,False,True,2.0,False,49.0,False,0.0,0.0,0.0
3,7,False,False,False,30.0,False,6.0,False,NaN,NaN,NaN
4,10207,False,False,False,NaN,False,82.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20625,35303,False,False,False,1.0,False,6.0,False,NaN,NaN,NaN
20626,35164,False,False,False,NaN,False,471.0,False,NaN,NaN,NaN
20627,35308,False,False,False,2.0,False,NaN,True,NaN,NaN,NaN
20628,35257,False,False,False,1.0,False,21.0,False,NaN,NaN,NaN


In [33]:
df_multiplayer_modes.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv\multiplayer_modes.csv', index=False)

GAME_MULTIPLAYER_MODE

In [38]:
def get_all_game_multiplayer_modes():
    get_all_game_multiplayer_modes = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/games', headers=headers , data=f'fields multiplayer_modes; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            get_all_game_multiplayer_modes.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break


    return pd.DataFrame(get_all_game_multiplayer_modes)

In [ ]:
df_game_multiplayer_mode = get_all_game_multiplayer_modes()


,id,multiplayer_modes
0,330684,NaN
1,177310,NaN
2,129194,NaN
3,282079,NaN
4,63844,NaN
...,...,...
323271,89219,NaN
323272,8993,NaN
323273,154420,NaN
323274,6253,NaN


In [40]:
df_game_multiplayer_mode = df_game_multiplayer_mode.explode("multiplayer_modes")

In [42]:
df_game_multiplayer_mode.to_csv(r'C:\Users\Martin\VS_code\Wild code school\data_game_new\data-game\csv\game_multiplayer_mode.csv', index=False)

PLATFORM_MULTIPLAYER_MODE

In [ ]:
def get_all_platform_multiplayer_mode():
    get_all_platform_multiplayer_mode = []
    offset = 0
    limit = 500  # Maximum limit for IGDB API

    while True:
        response = post('https://api.igdb.com/v4/multiplayer_modes', headers=headers , data=f'fields platform ; limit {limit}; offset {offset};')
        
        if response.status_code == 200:
            response_json = response.json()
            if not response_json:  # If no more data, break the loop
                break
            get_all_platform_multiplayer_mode.extend(response_json)
            offset += limit
        else:
            print("Error fetching data from the API")
            print("Status code:", response.status_code)
            break


    return pd.DataFrame(get_all_game_multiplayer_modes)